# Weekly Assignment 4 : Greenhouse Gas Emissions

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly as py

In [ ]:
# Load excel sheet
df = pd.read_excel('/Users/andrea/Desktop/Notebooks/DVS/datasets/Greenhouse gas emission per capita.xls',
                  skiprows=2,
                  skipfooter=9)

In [ ]:
# Drop unnecessary row and rename columns
df.columns = ['geo', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', 
              '2011', '2012', '2013', '2014', '2015', '2016', '2017']

df = df.drop(df.index[0])

In [ ]:
# Drop countries that will not appear on the map and reset index
excludes = ['Cyprus', 'Malta', 'Liechtenstein', 'Turkey'] 
df=df[~df['geo'].isin(excludes)]
df = df.reset_index(drop=True)

In [ ]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in df.iterrows(): 
    # Create list for the current row 
    my_list =[rows['2000'], rows['2001'], rows['2002'], rows['2003'], rows['2004'], rows['2005'],
                rows['2006'], rows['2007'], rows['2008'], rows['2009'], rows['2010'], rows['2011'], rows['2012'],
                rows['2013'], rows['2014'], rows['2015'], rows['2016'], rows['2017']]
      
    # append the list to the final list 
    Row_list.append(my_list)  

In [ ]:
# Create new dataframe with a "Country" column and one column 
# containing all yearly emissions for each country
data = {'Country': df.geo, 'Emission': Row_list}
new_df = pd.DataFrame(data=data)
new_df = new_df.explode('Emission').reset_index()
new_df = new_df.drop('index', axis=1)

a = list(range(2000, 2018))
b = np.tile(a, 29)
new_df['Year'] = b
new_df['Emission'] = new_df['Emission'].astype('float')

In [ ]:
# Create a new column to sort emissions into value bins
new_df['category'] = ''

def set_cat(row):
    if row['Emission'] > 0 and row['Emission'] < 5:
        return '1 - 4'
    if row['Emission'] >= 5 and row['Emission'] < 9:
        return '5 - 8'
    if row['Emission'] >= 9 and row['Emission'] < 13:
        return '9 - 12'
    if row['Emission'] >= 13 and row['Emission'] < 17:
        return '13 - 16'
    if row['Emission'] >= 17 and row['Emission'] < 21:
        return '17 - 20'
    if row['Emission'] >= 21 and row['Emission'] < 25:
        return '21 - 24'
    if row['Emission'] >= 25 and row['Emission'] < 29:
        return '25 - 28'
    if row['Emission'] >= 29 and row['Emission'] < 33:
        return '29 - 32'

new_df = new_df.assign(category=new_df.apply(set_cat, axis=1))

# Adds all available categories to each year
catg = new_df['category'].unique()
yrs = new_df['Year'].unique()

for yr in yrs:
    for i in catg:
        new_df = new_df.append({
            'Country' : 'M',
            'Emission' : 'N',
            'Year' : yr,
            'category' : i
        }, ignore_index=True)
        
new_df = new_df.mask(new_df.eq('None')).dropna()

In [ ]:
# Create choropleth map
fig = px.choropleth(new_df,
                    locations="Country",
                    color="category",
                    color_discrete_map={
                        '1 - 4': 'rgb(255,245,240)',
                        '5 - 8' : 'rgb(254,224,210)',
                        '9 - 12' : 'rgb(252,187,161)',
                        '13 - 16' : 'rgb(252,146,114)',
                        '17 - 20' : 'rgb(239,59,44)',
                        '21 - 24' : 'rgb(203,24,29)',
                        '25 - 28' : 'rgb(165,15,21)',
                        '29 - 32' : 'rgb(103,0,13)'},
                    category_orders={
                      'category' : [
                          '1 - 4',
                          '5 - 8',
                          '9 - 12',
                          '13 - 16',
                          '17 - 20',
                          '21 - 24',
                          '25 - 28',
                          '29 - 32'
                      ]
                    },
                    animation_frame="Year",
                    scope='europe',
                    hover_name='Country',
                    hover_data={'Emission' : True},
                    locationmode='country names',
                    )

# Adjust map layout styling
fig.update_layout(
    showlegend = True,
    legend_title_text='Emission',
    title_text = 'Greenhouse Gas Emissions for 29 European Countries'  +  
        '<span style="font-size: 12px;"> between 2000 and 2017</span>',
    plot_bgcolor='rgba(0,0,0,0)',
    geo = go.layout.Geo(
        scope = 'europe',
        showframe = False,
        showcoastlines = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "white" ,
        coastlinecolor = "white"),
    autosize=False,
    width=700,
    height=700,
)

# Show map
fig.show()